# **Aplicaciones en Tecnologías del Lenguaje - Práctica Temas 1 y 2**
## Notebook 4 - Results and Conclusions
### Student: Alvarez Lacasa, Lucas Alfredo

The goal of this final notebook is to put all results together and draw some conclusions from the experimentation carried out.

Instead of providing a PDF with this information, we will provide it here.

# Results

The summary with all of the best results obtained during our experimentation can be found in the following table.

---

| Notebook | Model                                         | Model Size | Dataset         | Representation                                          | Accuracy | Macro-Precision | Macro-Recall | Macro-F1   | Config Highlights |
|----------|----------------------------------------------|------------|----------------|--------------------------------------------------------|----------|----------------|--------------|------------|-------------------|
| 2        | Naive Bayes                                  |            | v1             | BOW: tf<br>Level: word<br>N-grams: (1.1)               | 0.6365   | 0.6368         | 0.6173       | 0.6128     | -                 |
| 2        | LogisticRegression                          |            | v1             | BOW: tf-idf<br>Level: word<br>N-grams: (1.2)           | 0.6755   | 0.6793         | 0.6587       | 0.6576     | (C=1000, max_iter=500, solver='liblinear') |
| 2        | LinearSVC                                   |            | v1             | BOW: tf-idf<br>Level: word<br>N-grams: (1.2)           | 0.6767   | 0.6796         | 0.6606       | **0.6599** | (C=10)            |
| 2        | RandomForests                               |            | v1             | BOW: tf-idf<br>Level: word<br>N-grams: (1.2)<br>Method: chi2 - 20k | 0.6018   | 0.6507         | 0.5620       | 0.5074     | (max_depth=64, n_estimators=256, n_jobs=-1, random_state=999) |
| 2        | XGBoost                                    |            | v1             | BOW: tf<br>Level: word<br>N-grams: (1.1)<br>Method: chi2 - 10k | 0.6369   | 0.6548         | 0.6091       | 0.5932     | (learning_rate=0.1, max_depth=64, n_estimators=256) |
| 3        | paraphrase-multilingual-MiniLM-L12-v2 Setfit | 118M       | v2 sampled 50k | Tokenizer<br>Vocab: ~ 250k tokens<br>Max Input: 128 tokens | 0.7271   | 0.7414         | 0.7100       | 0.7106     | (batch_size=64, num_iterations=1, num_epochs=1, max_steps=1563, add_early_stopping=False, early_stopping_patience=0) |
| 3        | distiluse-base-multilingual-cased-v1 Setfit  | 135M       | v3             | Tokenizer<br>Vocab: ~ 120k tokens<br>Max Input: 128 tokens | 0.7154   | 0.7130         | 0.7080       | 0.7093     | (batch_size=64, num_iterations=1, num_epochs=10, max_steps=18750, add_early_stopping=True, early_stopping_patience=2) |
| 3        | LaBSE Setfit                                | 471M       | v3             | Tokenizer<br>Vocab: ~ 501k tokens<br>Max Input: 256 tokens | 0.7272   | 0.7868         | 0.7005       | 0.6940     | (batch_size=32, num_iterations=1, num_epochs=3, max_steps=11250, add_early_stopping=False, early_stopping_patience=0) |
| 3        | paraphrase-multilingual-mpnet-base-v2       | 278M       | v3             | Tokenizer<br>Vocab: ~ 250k tokens<br>Max Input: 512 tokens | 0.7338   | 0.7579         | 0.7142       | **0.7140** | (batch_size=64, num_iterations=1, num_epochs=1, max_steps=1875, add_early_stopping=False, early_stopping_patience=0) |


---


_As mentioned in previous notebooks, for a better visualization, results will be left in the following [Google Spreadsheet](https://docs.google.com/spreadsheets/d/1mzY6YMFSP3kXgH3qgbJnX1w0f463ZQ06n32tLGgqOos/edit?usp=sharing)._



# Conclusions

## Recap
- We tackled the [IberAuTexTification challenge](https://sites.google.com/view/iberautextification/home?authuser=0) from `IberLEF 2024` to explore a real-world text classification task different than SA and NER. Specifically, the **MGT Detection task** aimed at distinguishing machine-generated from human-generated text.
- In `1_data_exploration`, we performed extensive data analysis and visualization. We experimented with multiple dataset versions with varying levels of cleanup, carefully examining the slight imbalance (with machine-generated texts slightly dominating) and exploring stratified sampling strategies.
- We developed four dataset variants, each applying different cleaning approaches. For contextual models, we deliberately limited the cleanup to preserve as much original information as possible.
- We began by establishing baselines with classic ML methods, using macro-averaged metrics to fairly assess performance given the binary class imbalance.
- We then moved on to DL approaches by fine-tuning SentenceTransformers with [SetFit](https://github.com/huggingface/setfit). Despite facing memory limitations early on, we eventually leveraged a sufficiently powerful AWS instance to complete training on the full dataset, although this did not yield significant improvements.

## Highlights

#### Classic ML Models vs. Transformer-Based Models
- The classic ML models trained with BoW representations showed **moderate performance**, with `LinearSVC` achieving the best **Macro-F1 score (65.99%)** among them.
- Among the classic models, `LogisticRegression` and `LinearSVC` performed similarly, slightly outperforming Naïve Bayes, while `Random Forests` and `XGBoost` struggled the most.
- `Random Forests` had the weakest performance (Macro-F1: 50.74%), likely due to its reliance on feature selection with Chi-Square filtering, which may have reduced useful information.
- In contrast, **Transformer-based models (SetFit fine-tuned models) outperformed traditional ML models significantly**, with the best-performing model being `paraphrase-multilingual-mpnet-base-v2` with **Macro-F1: 71.4%)**.

#### Impact of Fine-Tuning on Transformer-Based Models
- The SetFit models benefited from fine-tuning strategies, as shown by their higher Macro-F1 scores compared to classic models.
- Early stopping seems to have limited impact, as the best-performing models were trained without early stopping.
- Training for more epochs does not necessarily guarantee better performance. We also observed this, as the model tended to overfit to the training set and generalize worse for the validation set.

#### Model Size and Performance Trade-Off
- Larger models (e.g., LaBSE with 471M parameters) did not necessarily lead to better performance:
  - `LaBSE` had the highest macro-precision (78.68%) but lower macro-F1 (69.4%) than other models.
  - This indicates that while `LaBSE` may be precise when making predictions, it is **less balanced in recall**, meaning it misses some relevant cases.
- Smaller models like `paraphrase-multilingual-MiniLM-L12-v2` (118M) achieved comparable or even better performance with much fewer parameters, making them more efficient for productization.

#### Effect of Tokenization and Input Size
- The models using larger input sizes performed the best.
- This suggests that being able to capture longer contexts benefits classification accuracy. However, `paraphrase-multilingual-MiniLM-L12-v2` (128 tokens) was still competitive, indicating that fine-tuning played a bigger role than just increasing input size.

#### **Key Takeaways**
- 🥇 **`paraphrase-multilingual-mpnet-base-v2`**, with an accuracy of 73.38% and a **macro-F1 of 71.40%**, is the best performing model. This is a fairly small model as well, so you could put it in production even in CPU and get a decent performance.
- 🥈 Special mention for `LinearSVC` and `LogisticRegression`, which despite being ML-based approaches, still reach a decent performance of around **66% macro-f1**. Pretty sure we could enhance this with some extra features, as I mention in the "Pending Work" section.
- **We couldn't surpass the 71% macro-f1**. There seems to be some sort of limitation in the data or in the capability of generalization of the models that we train. As I mentioned in the first notebook, it's a bit weird that the test set evaluates on domains that are completely different from the ones we use to train the models. We tried to train with more and more data, but that didn't have a significant impact.
- Fine-tuning has a significant positive impact on performance, and smaller, efficiently trained models can outperform larger, more complex ones.

*Overall, while our results are respectable (a hypothetical ranking of 6th in the challenge), there remains substantial room for improvement—especially for deep learning approaches.*


## Pending Work
- Investigate under-sampling techniques to address class imbalance, complementing our current strategy of retaining as much data as possible.
- Explore the `prompt` field in the dataset for its potential value, especially in the context of multi-class classification.
- Evaluate zero-shot and few-shot performance using reasoning models (e.g., `DeepSeek-R1` or `OpenAI o3-mini-high`) to assess their ability to differentiate between human and machine-generated text.
- Consider lighter DL architectures (e.g., `RNNs`, `CNNs`) as alternatives to transformer-based models.
- Experiment with a broader range of SetFit hyperparameters, like optimizer selection and/or learning rate adjustments to potentially improve performance.
- ⭐ Incorporate hand-crafted features such as bigram/trigram probabilities (from a LM we can train) to provide additional cues for distinguishing human versus machine-generated text.
- ⭐ Conduct a thorough error analysis on the best-performing model to identify common failure cases and guide future improvements.